In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'churn-modelling:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F156197%2F358170%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240224%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240224T020717Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D37587987fddb7ab7c6f54f75da906bc2f1f56eb92099a42b8cfb7634310e4b90303b3e51ce0d8860104d59f94188252f66456e4721fa98f4dcda120f1fe95937d330893b3938003e609fd85af767b4686eddf45628eb4cac4362917ca28f3e73fb15e07afb15f31e9954e0543641228ee61b55ff8aafb0fd61a4cbb0c545dbbcfd1e028e20a9b6b6013f453b31c0ae115624edf98c5567ab1effb61ee7cf648a2088ad9c5d6b9eaa1e7473459b77eae413cc5de3e1ec9a4642011b4b1ea49b4b7926156e959c41e9713d04610e8445f358b54b866fc6fed5faf26b6755e2ff87a8507609f9d0b494237309a025719a7946e0d5ec20c52adad21716040f252bc6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/kaggle/input/churn-modelling/Churn_Modelling.csv')
df.head()

In [ ]:
#Drop the columns which is not effective in knowing whether a customer will leave or not based upon domain understanding

df1 = df.drop(['RowNumber','CustomerId','Surname'] , axis='columns')
df1.head()

In [ ]:
df1.columns

**EDA**

In [ ]:
import seaborn as sns

In [ ]:
d1 = df1[['CreditScore','Age','Balance','EstimatedSalary','Exited']]

sns.pairplot(d1 , hue = 'Exited')

In [ ]:
df1['Gender'].value_counts()

In [ ]:
df1.groupby('Gender')['Exited'].value_counts().unstack()

In [ ]:
df1.groupby('Gender')['Exited'].value_counts().unstack().plot.bar()

In [ ]:
exit_count = df1['Exited'].value_counts()

plt.pie(exit_count, autopct = "%1.1f%%",explode=[0,0.1],labels = exit_count.index)
plt.title("Customer Exit Distribution")
plt.show()

In [ ]:
df1['Geography'].value_counts()

In [ ]:
geo_exit = df1.groupby('Geography')['Exited'].value_counts().unstack()
geo_exit

In [ ]:
geo_exit.plot.bar()

In [ ]:
ax = sns.countplot(x = 'Geography' , data = df1 , hue = 'Exited')
for value in ax.patches:
    percentage = ((value.get_height()/len(df))*100)
    x = value.get_x() + value.get_width()/2 - 0.05
    y = value.get_y() + value.get_height() + 50
    ax.annotate(f"{round(percentage,2)} %" , (x,y))

plt.show()

In [ ]:
#Custmer churn by HasCard

ax = sns.countplot(x = 'HasCrCard' , data = df1 , hue = 'Exited')
for value in ax.patches:
    percentage = ((value.get_height()/len(df))*100)
    x = value.get_x() + value.get_width()/2 - 0.05
    y = value.get_y() + value.get_height() + 50
    ax.annotate(f"{percentage} %" , (x,y))

plt.show()

In [ ]:
#Custmer churn by Number of Products

ax = sns.countplot(x = 'NumOfProducts' , data = df1 , hue = 'Exited')
for value in ax.patches:
    percentage = ((value.get_height()/len(df))*100)
    x = value.get_x() + value.get_width()/2 - 0.05
    y = value.get_y() + value.get_height() + 50
    ax.annotate(f"{round(percentage,2)} %" , (x,y))

plt.show()

In [ ]:
#Custmer churn by IsActivemember

ax = sns.countplot(x = 'IsActiveMember' , data = df1 , hue = 'Exited')
for value in ax.patches:
    percentage = ((value.get_height()/len(df))*100)
    x = value.get_x() + value.get_width()/2 - 0.05
    y = value.get_y() + value.get_height() + 50
    ax.annotate(f"{round(percentage,2)} %" , (x,y))

plt.show()

In [ ]:
#Custmer churn by IsActivemember

plt.figure(figsize=(10,6))
ax = sns.countplot(x = 'Tenure' , data = df1 , hue = 'Exited')
for value in ax.patches:
    percentage = ((value.get_height()/len(df))*100)
    x = value.get_x() + value.get_width()/2 - 0.05
    y = value.get_y() + value.get_height() + 20
    ax.annotate(f"{round(percentage,1)} %" , (x,y) , rotation=90)

plt.show()

In [ ]:
#Salary & Exit
sns.histplot(x='EstimatedSalary' , data = df1 , hue = 'Exited')
plt.show()

In [ ]:
df1.head()

In [ ]:
sns.histplot(x='CreditScore' , data = df1 , hue = 'Exited')
plt.show()

In [ ]:
for idx,group in df1.groupby('Exited')['CreditScore']:
    #print(sns.boxplot(group['CreditScore']))
    plt.subplot(1,2,idx+1)
    plt.title("Credit Score")
    plt.boxplot(group)
    plt.xlabel(idx)
    plt.tight_layout()
plt.show()

In [ ]:
sns.histplot(x='Age' , data = df1 , hue = 'Exited')
plt.show()

In [ ]:
for idx,group in df1.groupby('Exited')['Age']:
    #print(sns.boxplot(group['CreditScore']))
    plt.subplot(1,2,idx+1)
    plt.title("Age")
    plt.boxplot(group)
    plt.xlabel(idx)
    plt.tight_layout()
plt.show()

In [ ]:
sns.histplot(x='Balance' , data = df1 , hue = 'Exited')
plt.show()

In [ ]:
for idx,group in df1.groupby('Exited')['Balance']:
    #print(sns.boxplot(group['CreditScore']))
    plt.subplot(1,2,idx+1)
    plt.title("Balance")
    plt.boxplot(group)
    plt.xlabel(idx)
    plt.tight_layout()
plt.show()

**Model Building**

In [ ]:
df1.isnull().sum()

In [ ]:
df1.dtypes

**One Hot Encoding for Non-Numerical Data Columns**

In [ ]:
df2 = pd.get_dummies(df1)
df2.head()

**Generate Features and Lables**

In [ ]:
X = df2.drop('Exited', axis = 'columns')
Y = df2['Exited']

**Over Sampling As Data is Unbalanced - Category 1 has only 20% of total data distribution**

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()

Xr,Yr = ros.fit_resample(X,Y)

In [ ]:
Yr.value_counts()

**Import Required Libraries**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

**Create the model**

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(Xr,Yr,train_size=0.75)
model1 = LogisticRegression()

model1.fit(xtrain,ytrain)

**Model Performance**

In [ ]:
ytrainPred = model1.predict(xtrain)
ytestPred = model1.predict(xtest)

In [ ]:
print("Training Data Accuracy", (ytrain == ytrainPred).sum()/len(xtrain))
print("Test Data Accuracy", (ytest == ytestPred).sum()/len(xtest))

**Confusion MAtrix**

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
conf_matrix_train = pd.DataFrame(confusion_matrix(ytrain , ytrainPred))
conf_matrix_train

In [ ]:
conf_matrix_train/conf_matrix_train.sum(axis=1)

In [ ]:
conf_matrix_test = pd.DataFrame(confusion_matrix(ytest , ytestPred))
conf_matrix_test

In [ ]:
conf_matrix_test/conf_matrix_test.sum(axis=1)

**ROC AUC Curve Analysis - to fix Unbalanced data distribution**

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,train_size=0.75)
model2 = LogisticRegression()

model2.fit(xtrain,ytrain)

ytrainPred = model2.predict(xtrain)
ytestPred = model2.predict(xtest)

print("Training Data Accuracy", (ytrain == ytrainPred).sum()/len(xtrain))
print("Test Data Accuracy", (ytest == ytestPred).sum()/len(xtest))

In [ ]:
conf_matrix_train = pd.DataFrame(confusion_matrix(ytrain , ytrainPred))
print(conf_matrix_train)
print()
conf_matrix_train/conf_matrix_train.sum(axis=1)

In [ ]:
conf_matrix_test = pd.DataFrame(confusion_matrix(ytest , ytestPred))
print(conf_matrix_test)
print()
conf_matrix_test/conf_matrix_test.sum(axis=1)

In [ ]:
ypredTrainProb = model2.predict_proba(xtrain)[:,1]

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score

auc = roc_auc_score(ytrain,ypredTrainProb)
print(auc)

In [ ]:
fpr,tpr,thresh = roc_curve(ytrain,ypredTrainProb)

In [ ]:
print(fpr)

In [ ]:
print(tpr)

In [ ]:
print(thresh)

In [ ]:
g_val = tpr * (1 - fpr)
print(g_val)

In [ ]:
print(max(g_val) , np.argmax(g_val))

In [ ]:
print(f"Thresh for max tpr & min fpr = {thresh[np.argmax(g_val)]}")

In [ ]:
print(f"TPR or Recall or Sensitivity class 1 Accuracy for best thresh = {tpr[np.argmax(g_val)]}")

In [ ]:
print(f"FPR for best thresh = {fpr[np.argmax(g_val)]}")

In [ ]:
print(f"TNR or Specificity or class 0 Accuracy for best thresh = {1 - fpr[np.argmax(g_val)]}")

In [ ]:
plt.figure(figsize = (10,6))

plt.plot(fpr , tpr , marker = '*')

plt.show()

**Evalauting model performance if threshold to decide 1 or 0 is 0.21**